In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cd /content/drive/MyDrive

In [4]:
!git clone https://github.com/yannikc/ML4H_project2.git

Cloning into 'ML4H_project2'...
remote: Enumerating objects: 13329, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 13329 (delta 16), reused 51 (delta 10), pack-reused 13270 (from 1)
Receiving objects: 100% (13329/13329), 2.09 GiB | 17.74 MiB/s, done.
Resolving deltas: 100% (1264/1264), done.
Updating files: 100% (23501/23501), done.


In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from os import listdir
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from os import listdir
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

def permute_labels(dataset):
  # 1. Copy the original targets
  old_targets = dataset.targets.copy()

  # 2. Permute them in-place
  random.shuffle(old_targets)

  # 3. Assign the shuffled labels back
  dataset.targets = old_targets

  dataset.samples = [
      (path, new_label)
      for (path, _), new_label in zip(dataset.samples, dataset.targets)
  ]
  return dataset

trainset = ImageFolder(root="ML4H_project2/data_part2_processed/train/", transform=transform)
validationset = ImageFolder(root="ML4H_project2/data_part2_processed/val/", transform=transform)
testset = ImageFolder(root="ML4H_project2/data_part2_processed/test/", transform=transform)

trainset = permute_labels(trainset)
validationset= permute_labels(validationset)
testset= permute_labels(testset)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validationloader = DataLoader(validationset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=True)
images, labels = next(iter(trainloader))
print(images.shape)  # should be [32, 1, 1000, 1000]
print(labels.shape)  # should be [32]

torch.Size([32, 1, 1000, 1000])
torch.Size([32])


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return torch.sigmoid(x)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

In [8]:
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(trainloader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in validationloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(validationloader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

100%|██████████| 163/163 [01:41<00:00,  1.60it/s]


Epoch [1/10], Training Loss: 0.5821, Validation Loss: 0.7493


100%|██████████| 163/163 [01:42<00:00,  1.60it/s]


Epoch [2/10], Training Loss: 0.5781, Validation Loss: 0.8451


100%|██████████| 163/163 [01:41<00:00,  1.60it/s]


...

Epoch [8/10], Training Loss: 0.5721, Validation Loss: 0.8275


100%|██████████| 163/163 [01:41<00:00,  1.61it/s]


Epoch [9/10], Training Loss: 0.5713, Validation Loss: 0.8490


100%|██████████| 163/163 [01:44<00:00,  1.56it/s]


Epoch [10/10], Training Loss: 0.5715, Validation Loss: 0.8428


In [9]:
# Evaluation on test set
model.eval()
test_loss = 0.0
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        test_loss += loss.item()

        # Accuracy calculation
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels.unsqueeze(1)).sum().item()
        total += labels.size(0)

        # AUROC and AUPRC calculation
        all_preds.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(testloader)
accuracy = correct / total
auroc = roc_auc_score(all_labels, all_preds)
auprc = average_precision_score(all_labels, all_preds)

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy*100:.2f}%")
print(f"Test AUROC: {auroc:.4f}, Test AUPRC: {auprc:.4f}")

100%|██████████| 20/20 [00:09<00:00,  2.17it/s]

Test Loss: 0.6999, Test Accuracy: 62.50%
Test AUROC: 0.5246, Test AUPRC: 0.6295


In [10]:
# Saving the model weights
torch.save(model.state_dict(), 'randomized_CNN_weights.pth')